In [ ]:
!pip install Flask pyngrok line-bot-sdk requests --quiet
!pip install google-genai --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 818.9/818.9 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 165.6/165.6 kB 3.5 MB/s eta 0:00:00


In [ ]:
from google.colab import userdata

ngrok_authtoken = userdata.get('NGROK_AUTHTOKEN')
line_channel_access_token = userdata.get('LINE_CHANNEL_ACCESS_TOKEN')
line_channel_secret = userdata.get('LINE_CHANNEL_SECRET')
gemini_api_key = userdata.get('GEMINI_API_KEY')
port = 5051

In [ ]:
import os
from pyngrok import ngrok

In [ ]:
ngrok.kill()

In [ ]:
import requests

ngrok.set_auth_token(ngrok_authtoken)
tunnel = ngrok.connect(5051, name="linebot_tunnel")
webhook_url = tunnel.public_url

print(f"Ngrok URL: {webhook_url}")

# 自動更新 LINE Webhook URL
def update_line_webhook(webhook_url):
    """使用 LINE Messaging API 更新 Webhook URL"""
    url = "https://api.line.me/v2/bot/channel/webhook/endpoint"
    headers = {
        "Authorization": f"Bearer {line_channel_access_token}",
        "Content-Type": "application/json"
    }
    data = {
        "endpoint": webhook_url
    }

    response = requests.put(url, headers=headers, json=data)

    if response.status_code == 200:
        print(f"✅ LINE Webhook URL 已自動更新為：{webhook_url}")
        return True
    else:
        print(f"❌ 更新失敗：{response.status_code} - {response.text}")
        return False

update_line_webhook(webhook_url)

Ngrok URL: https://whistleable-lupe-perfunctory.ngrok-free.dev
✅ LINE Webhook URL 已自動更新為：https://whistleable-lupe-perfunctory.ngrok-free.dev


True

In [ ]:
from google import genai
from google.genai.types import Tool, GenerateContentConfig, GoogleSearch

client = genai.Client(api_key=gemini_api_key)

google_search_tool = Tool(
   google_search=GoogleSearch()
)

chat = client.chats.create(
    model="gemini-2.5-flash",
    config=GenerateContentConfig(
        system_instruction="你是一個中文的AI助手，請用繁體中文回答",
        tools=[google_search_tool],
        response_modalities=["TEXT"],
    )
)

In [ ]:
def stateful_query(payload):
    response = chat.send_message(message=payload)
    return response.text

In [ ]:
result = stateful_query("簡介明新科技大學")
print(result)

明新科技大學（Minghsin University of Science and Technology），簡稱明新科大，是一所位於台灣新竹縣新豐鄉的私立科技大學。學校的創立可追溯至1966年3月1日以「明新工業專科學校」立案，初期設有機械、土木、工業管理三科，並招收三百名新生。之後，學校歷經多次改制與升格，於1997年7月奉教育部核准改制為「明新技術學院」，並附設專科部；最終在2002年8月（或9月）獲教育部核准，優先升格為「明新科技大學」。2018年12月，學校更名為「明新學校財團法人明新科技大學」，展現其全新的發展格局。

明新科大的校名「明新」取自《大學》「在明明德，在新民，在止於至善」的精義，旨在闡揚人類與生俱來的德性與情操，並以培養具備高尚品德、專業學問與優良技術的學生為最終目標。學校的教育目標是培育「跨域整合、務實創新與全人學習」的專業人才，並以成為「一流產業大學」為發展願景。

學校現有半導體學院、工程學院、管理學院、民生學院、人文與設計學院以及共同教育學院等六個學院。其中，半導體學院是其重要特色之一，明新科大積極與國內外學術單位合作，推動半導體國際人才培育計畫，並在半導體產業人才培育方面表現突出，獲得企業肯定。

明新科大以其「產學合一」的辦學特色聞名，積極強化產學鏈結，提供契合產業需求的人才課程，使畢業生能「立即就業」。根據2022年《遠見》雜誌的調查，明新科大在「企業最愛公私立技職科大調查」的「四大領域」中獲得兩項企業最愛，其畢業生起薪在私立學校中排名第一，甚至優於部分國立科技大學。學校也積極推動「永續智慧商務」，建置相關教學與實習場域，並導入AI技術，發展智慧零售、智慧金融、智慧製造與智慧商業等實驗室。

學校位於新竹縣新豐鄉，佔地超過三十公頃，鄰近新竹科學園區與新竹工業區，享有豐富的產業資源，交通便利。明新科大畢業校友已逾九萬餘人，在各行各業中擔任中堅力量，廣受社會肯定。


In [ ]:
result2 = stateful_query("校長是誰？")
print(result2)

明新科技大學現任校長為呂明峯教授。他於2025年2月1日正式上任，成為明新科大的第11任校長。呂明峯校長在半導體教育和產業實務方面擁有豐富經驗，曾擔任電子工程系主任、研發長、工學院院長、半導體學院院長以及產學長等職務。他以「四大核心模組」作為校務治理策略，旨在將明新科大打造成具國際魅力的產業大學。


In [ ]:
from flask import Flask, request, abort
import logging
import os
import time
from google.genai import types

from linebot.v3 import (
    WebhookHandler
)
from linebot.v3.exceptions import (
    InvalidSignatureError
)
from linebot.v3.messaging import (
    Configuration,
    ApiClient,
    MessagingApi,
    MessagingApiBlob,
    ReplyMessageRequest,
    TextMessage
)
from linebot.v3.webhooks import (
    MessageEvent,
    TextMessageContent,
    FileMessageContent
)

app = Flask(__name__)

logging.basicConfig(
    level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s"
)
app.logger.setLevel(logging.INFO)

configuration = Configuration(access_token=line_channel_access_token)
handler = WebhookHandler(line_channel_secret)


UPLOAD_DIR = "/content/uploaded_files"
os.makedirs(UPLOAD_DIR, exist_ok=True)


user_sessions = {}

def get_user_session(user_id):
    """取得或建立使用者的對話 session"""
    if user_id not in user_sessions:

        new_chat = client.chats.create(
            model="gemini-2.5-flash",
            config=GenerateContentConfig(
                system_instruction="你是一個中文的AI助手，請用繁體中文回答。如果使用者有提供參考文件，請根據文件內容回答問題。",
                tools=[google_search_tool],
                response_modalities=["TEXT"],
            )
        )
        user_sessions[user_id] = {
            "chat": new_chat,
            "uploaded_file": None
        }
    return user_sessions[user_id]

def download_line_file(message_id, file_name):
    """從 LINE 下載使用者上傳的檔案"""
    with ApiClient(configuration) as api_client:
        line_bot_blob_api = MessagingApiBlob(api_client)
        file_content = line_bot_blob_api.get_message_content(message_id)

        file_path = os.path.join(UPLOAD_DIR, file_name)

        with open(file_path, 'wb') as f:
            f.write(file_content)

        return file_path

def upload_file_to_gemini(file_path):
    """上傳檔案到 Gemini Files API"""
    uploaded_file = client.files.upload(
        file=file_path,
        config={'display_name': os.path.basename(file_path)}
    )

    while uploaded_file.state.name == "PROCESSING":
        print("檔案處理中...")
        time.sleep(1)
        uploaded_file = client.files.get(name=uploaded_file.name)

    if uploaded_file.state.name == "FAILED":
        raise Exception("檔案上傳處理失敗")

    return uploaded_file

def query_with_rag(user_id, question):
    """使用 RAG 模式回答問題"""
    session = get_user_session(user_id)
    uploaded_file = session["uploaded_file"]

    if uploaded_file:
        response = client.models.generate_content(
            model="gemini-2.5-flash",
            contents=[
                types.Content(
                    role="user",
                    parts=[
                        types.Part.from_uri(
                            file_uri=uploaded_file.uri,
                            mime_type=uploaded_file.mime_type
                        ),
                        types.Part.from_text(text=f"請根據上述提供的檔案內容，用繁體中文回答這個問題：{question}")
                    ]
                )
            ]
        )
        return response.text
    else:
        response = session["chat"].send_message(message=question)
        return response.text

@app.route("/", methods=['POST'])
def callback():
    signature = request.headers['X-Line-Signature']
    body = request.get_data(as_text=True)
    print("BODY: ", body)
    app.logger.info("Request body: " + body)

    try:
        handler.handle(body, signature)
    except InvalidSignatureError:
        app.logger.info("Invalid signature.")
        abort(400)

    return 'OK'

@handler.add(MessageEvent, message=TextMessageContent)
def handle_message(event):
    """處理文字訊息"""
    text = event.message.text
    user_id = event.source.user_id

    with ApiClient(configuration) as api_client:
        line_bot_api = MessagingApi(api_client)

        if text.startswith('AI '):
            prompt = text[3:]
            try:
                reply_text = query_with_rag(user_id, prompt)

                session = get_user_session(user_id)
                if session["uploaded_file"]:
                    reply_text = f"📄 [RAG 模式]\n\n{reply_text}"

                line_bot_api.reply_message_with_http_info(
                    ReplyMessageRequest(
                        reply_token=event.reply_token,
                        messages=[TextMessage(text=reply_text)]
                    )
                )
            except Exception as e:
                line_bot_api.reply_message_with_http_info(
                    ReplyMessageRequest(
                        reply_token=event.reply_token,
                        messages=[TextMessage(text=f"❌ 發生錯誤：{str(e)}")]
                    )
                )
        elif text == "清除文件":
            session = get_user_session(user_id)
            session["uploaded_file"] = None
            line_bot_api.reply_message_with_http_info(
                ReplyMessageRequest(
                    reply_token=event.reply_token,
                    messages=[TextMessage(text="✅ 已清除上傳的文件，恢復一般對話模式。")]
                )
            )
        else:
            line_bot_api.reply_message_with_http_info(
                ReplyMessageRequest(
                    reply_token=event.reply_token,
                    messages=[TextMessage(text="請輸入「AI 問題」來開始對話\n或上傳 TXT/PDF 檔案啟用 RAG 模式")]
                )
            )

@handler.add(MessageEvent, message=FileMessageContent)
def handle_file_message(event):
    """處理使用者上傳的檔案"""
    user_id = event.source.user_id
    file_name = event.message.file_name

    with ApiClient(configuration) as api_client:
        line_bot_api = MessagingApi(api_client)

        if not (file_name.endswith('.txt') or file_name.endswith('.pdf')):
            line_bot_api.reply_message_with_http_info(
                ReplyMessageRequest(
                    reply_token=event.reply_token,
                    messages=[TextMessage(text="⚠️ 目前只支援 TXT 或 PDF 檔案")]
                )
            )
            return

        try:
            file_path = download_line_file(event.message.id, file_name)
            print(f"檔案已下載：{file_path}")

            uploaded_file = upload_file_to_gemini(file_path)
            print(f"檔案已上傳到 Gemini：{uploaded_file.uri}")

            session = get_user_session(user_id)
            session["uploaded_file"] = uploaded_file

            line_bot_api.reply_message_with_http_info(
                ReplyMessageRequest(
                    reply_token=event.reply_token,
                    messages=[TextMessage(text=f"✅ 檔案「{file_name}」上傳成功！\n\n現在您可以輸入「AI 問題」來詢問關於這份文件的問題。\n\n輸入「清除文件」可恢復一般對話模式。")]
                )
            )
        except Exception as e:
            line_bot_api.reply_message_with_http_info(
                ReplyMessageRequest(
                    reply_token=event.reply_token,
                    messages=[TextMessage(text=f"❌ 檔案處理失敗：{str(e)}")]
                )
            )

if __name__ == "__main__":
    app.run(port=port)

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5051
INFO:werkzeug:Press CTRL+C to quit
INFO:__main__:Request body: {"destination":"U290cce7d33943b09de06b23f4e8d3f6c","events":[{"type":"message","message":{"type":"file","id":"595683524410343680","markAsReadToken":"CugQn2yJ4sQ62YkeVwM8JPLLuU_2pTiHSU4Uh0kFZ_Lo-Lp2FaDx4TNOltBgbsuhXXvVGbf5f_UPS6ivv5xuEe88oEJX_7VfDob_zd-knkdMmChmNzLMdbgPNSa0ARgvXuOgIACN4VK_U0iZn5iubzq72JN-WKPoeCShqT3Wtg7FvXbszLxNuoO1cMHQmOWT9diIJ7gX7Xwwn81YSiAYsg","fileName":"faq.txt","fileSize":20,"contentProvider":{"type":"line"}},"webhookEventId":"01KEF3SWJFXCJWAF2DAY4QY384","deliveryContext":{"isRedelivery":false},"timestamp":1767886352746,"source":{"type":"user","userId":"U9bce42694348bd9a01e5e0f0f4f36da9"},"replyToken":"a6dd93faa337491bb89635e67ee7bb98","mode":"active"}]}


BODY:  {"destination":"U290cce7d33943b09de06b23f4e8d3f6c","events":[{"type":"message","message":{"type":"file","id":"595683524410343680","markAsReadToken":"CugQn2yJ4sQ62YkeVwM8JPLLuU_2pTiHSU4Uh0kFZ_Lo-Lp2FaDx4TNOltBgbsuhXXvVGbf5f_UPS6ivv5xuEe88oEJX_7VfDob_zd-knkdMmChmNzLMdbgPNSa0ARgvXuOgIACN4VK_U0iZn5iubzq72JN-WKPoeCShqT3Wtg7FvXbszLxNuoO1cMHQmOWT9diIJ7gX7Xwwn81YSiAYsg","fileName":"faq.txt","fileSize":20,"contentProvider":{"type":"line"}},"webhookEventId":"01KEF3SWJFXCJWAF2DAY4QY384","deliveryContext":{"isRedelivery":false},"timestamp":1767886352746,"source":{"type":"user","userId":"U9bce42694348bd9a01e5e0f0f4f36da9"},"replyToken":"a6dd93faa337491bb89635e67ee7bb98","mode":"active"}]}
檔案已下載：/content/uploaded_files/faq.txt
檔案已上傳到 Gemini：https://generativelanguage.googleapis.com/v1beta/files/830c0u6jy7d1


INFO:werkzeug:127.0.0.1 - - [08/Jan/2026 15:32:35] "POST / HTTP/1.1" 200 -
INFO:__main__:Request body: {"destination":"U290cce7d33943b09de06b23f4e8d3f6c","events":[{"type":"message","message":{"type":"text","id":"595683626348708232","quoteToken":"9fjVM5QhlNBiOWjgMhQCaQeNrCCJJFt549LFy4R-gCiWEvKiuo2junstgZ-3E78H0LEcanUXKpxvlqj5fabZpZVwCwLyqi46TkQTdd4KvUAAoreeHLdlqteiZoA-yY5k9tBNjECFUHuVnwMBuyhh-w","markAsReadToken":"fo_PV6iPo3UDWickZ0O-I4r-nz30tuOZigazERxMUWbJke04fmnvXcNl2nuoOk5WttYHts-6ompqEpmhrpAx1UAD81FIwFUubVBWQ-VT2ODNZ9s5NVjZH5e1ylzvB1rInOIHhyO28lJaQ952h6EExTJOtbZSLl5su6ilwkKzj121mbqEJDo2N9Ysa_dqTiixicHqmUrnme0wgb7Zf9n8BQ","text":"AI what did mike have?"},"webhookEventId":"01KEF3VR57SXBQWJ0CN63GC066","deliveryContext":{"isRedelivery":false},"timestamp":1767886413482,"source":{"type":"user","userId":"U9bce42694348bd9a01e5e0f0f4f36da9"},"replyToken":"12dd5c3556e844fd8ba8c22900c87b20","mode":"active"}]}


BODY:  {"destination":"U290cce7d33943b09de06b23f4e8d3f6c","events":[{"type":"message","message":{"type":"text","id":"595683626348708232","quoteToken":"9fjVM5QhlNBiOWjgMhQCaQeNrCCJJFt549LFy4R-gCiWEvKiuo2junstgZ-3E78H0LEcanUXKpxvlqj5fabZpZVwCwLyqi46TkQTdd4KvUAAoreeHLdlqteiZoA-yY5k9tBNjECFUHuVnwMBuyhh-w","markAsReadToken":"fo_PV6iPo3UDWickZ0O-I4r-nz30tuOZigazERxMUWbJke04fmnvXcNl2nuoOk5WttYHts-6ompqEpmhrpAx1UAD81FIwFUubVBWQ-VT2ODNZ9s5NVjZH5e1ylzvB1rInOIHhyO28lJaQ952h6EExTJOtbZSLl5su6ilwkKzj121mbqEJDo2N9Ysa_dqTiixicHqmUrnme0wgb7Zf9n8BQ","text":"AI what did mike have?"},"webhookEventId":"01KEF3VR57SXBQWJ0CN63GC066","deliveryContext":{"isRedelivery":false},"timestamp":1767886413482,"source":{"type":"user","userId":"U9bce42694348bd9a01e5e0f0f4f36da9"},"replyToken":"12dd5c3556e844fd8ba8c22900c87b20","mode":"active"}]}


INFO:werkzeug:127.0.0.1 - - [08/Jan/2026 15:33:36] "POST / HTTP/1.1" 200 -
INFO:__main__:Request body: {"destination":"U290cce7d33943b09de06b23f4e8d3f6c","events":[{"type":"message","message":{"type":"text","id":"595683689867248161","quoteToken":"EVaS97uf32dYcgUNNnqKuQ46Wa5J8muzV1hZU_hCOJMiyk9tA7N1yB0DcjMOSIKwmziDE5hr6ZflPa5UtBH_9vImSu7-eKA4WdpAD_uqHgOx9bRKwEHhAqgFWc90SNc9JGucHxiXBSAfgSmYaEtPfQ","markAsReadToken":"RTECWAXfiPUn59lDfy45044BtWGAhT93t3QZiMHF6F5HMHcoBuNI20xLy5LINJAAoJBHVvRVD6Wh15dE5LKcmFFXsYExP2wMh6bALIlFnlxB2bqL9Y0E_yc7cIXAni8ZqXt72VjqQ5bCQYqw0rhJBWduP-e1i151Tde7fPtTwvxpdN6XeheY5A9lgGJFxONuRQ7TmyoKTXL4Ibz4rqEqlQ","text":"AI what did mike have"},"webhookEventId":"01KEF3WWWN6YTGF37YK6EPVZH8","deliveryContext":{"isRedelivery":false},"timestamp":1767886451344,"source":{"type":"user","userId":"U9bce42694348bd9a01e5e0f0f4f36da9"},"replyToken":"a6acd3391bd14a8883e0de807b5a46e8","mode":"active"}]}


BODY:  {"destination":"U290cce7d33943b09de06b23f4e8d3f6c","events":[{"type":"message","message":{"type":"text","id":"595683689867248161","quoteToken":"EVaS97uf32dYcgUNNnqKuQ46Wa5J8muzV1hZU_hCOJMiyk9tA7N1yB0DcjMOSIKwmziDE5hr6ZflPa5UtBH_9vImSu7-eKA4WdpAD_uqHgOx9bRKwEHhAqgFWc90SNc9JGucHxiXBSAfgSmYaEtPfQ","markAsReadToken":"RTECWAXfiPUn59lDfy45044BtWGAhT93t3QZiMHF6F5HMHcoBuNI20xLy5LINJAAoJBHVvRVD6Wh15dE5LKcmFFXsYExP2wMh6bALIlFnlxB2bqL9Y0E_yc7cIXAni8ZqXt72VjqQ5bCQYqw0rhJBWduP-e1i151Tde7fPtTwvxpdN6XeheY5A9lgGJFxONuRQ7TmyoKTXL4Ibz4rqEqlQ","text":"AI what did mike have"},"webhookEventId":"01KEF3WWWN6YTGF37YK6EPVZH8","deliveryContext":{"isRedelivery":false},"timestamp":1767886451344,"source":{"type":"user","userId":"U9bce42694348bd9a01e5e0f0f4f36da9"},"replyToken":"a6acd3391bd14a8883e0de807b5a46e8","mode":"active"}]}


INFO:werkzeug:127.0.0.1 - - [08/Jan/2026 15:34:14] "POST / HTTP/1.1" 200 -
